# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parametrized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## Layout of the SageMaker ModelBuild Project Template

The template provides a starting point for bringing your SageMaker Pipeline development to production.

```
|-- codebuild-buildspec.yml
|-- CONTRIBUTING.md
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
|-- README.md
|-- sagemaker-pipelines-project.ipynb
|-- setup.cfg
|-- setup.py
|-- tests
|   `-- test_pipelines.py
`-- tox.ini
```

A description of some of the artifacts is provided below:
<br/><br/>
Your codebuild execution instructions:
```
|-- codebuild-buildspec.yml
```
<br/><br/>
Your pipeline artifacts, which includes a pipeline module defining the required `get_pipeline` method that returns an instance of a SageMaker pipeline, a preprocessing script that is used in feature engineering, and a model evaluation script to measure the Mean Squared Error of the model that's trained by the pipeline:

```
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py

```
<br/><br/>
Utility modules for getting pipeline definition jsons and running pipelines:

```
|-- pipelines
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
```
<br/><br/>
Python package artifacts:
```
|-- setup.cfg
|-- setup.py
```
<br/><br/>
A stubbed testing module for testing your pipeline as you develop:
```
|-- tests
|   `-- test_pipelines.py
```
<br/><br/>
The `tox` testing framework configuration:
```
`-- tox.ini
```

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](img/pipeline-full.png)

### Getting some constants

We get some constants from the local execution environment.

In [39]:
%store -r sagemaker_project_id
%store -r sagemaker_project_name

In [41]:
print(sagemaker_project_id)

p-sdlwcai2qbn5


In [42]:
print(sagemaker_project_name)

chris-16086629152853634


In [40]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name


# Change these to reflect your project/business name or if you want to separate ModelPackageGroup/Pipeline from the rest of your team

model_package_group_name = sagemaker_project_id
pipeline_name = sagemaker_project_id

### Get the pipeline instance

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [ ]:
########### 
# Note:  I found this code in _utils.py that we may need to reverse-engineer

# def get_pipeline_driver(module_name, passed_args=None):
#     """Gets the driver for generating your pipeline definition.

#     Pipeline modules must define a get_pipeline() module-level method.

#     Args:
#         module_name: The module name of your pipeline.
#         passed_args: Optional passed arguments that your pipeline may be templated by.

#     Returns:
#         The SageMaker Workflow pipeline.
#     """
#     _imports = __import__(module_name, fromlist=["get_pipeline"])
#     kwargs = convert_struct(passed_args)
#     return _imports.get_pipeline(**kwargs)


# def convert_struct(str_struct=None):
#     return ast.literal_eval(str_struct) if str_struct else {}


In [43]:
from pipelines.dsoaws.pipeline import get_pipeline

pipeline = get_pipeline(
    region=region,
    role=role,
    bucket=bucket, # Note: in the stock project template examples, this is `default_bucket`.  We changed to match our convention of just `bucket`
    pipeline_name=pipeline_name,
    model_package_group_name=model_package_group_name,
    base_job_prefix=model_package_group_name
)

s3://sagemaker-us-east-1-231218423789/amazon-reviews-pds/tsv/
763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.1.0-cpu-py3
os.listdir(BASE_DIR): ['__init__.py', '.ipynb_checkpoints', '__pycache__', 'pipeline.py', 'tf_bert_reviews.py', 'preprocess-scikit-text-to-bert.py', 'src', 'inference.py', 'preprocess.py.orig']
763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.1.0-cpu


# Start Pipeline Execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

# IGNORE THE FOLLOWING ERROR:
```
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
```

In [44]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/chris-16086629152853634',
 'ResponseMetadata': {'RequestId': 'accb69ee-6cfc-4a16-9053-cf6835185c62',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'accb69ee-6cfc-4a16-9053-cf6835185c62',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Tue, 22 Dec 2020 18:52:16 GMT'},
  'RetryAttempts': 0}}

# Start Pipeline Execution

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [45]:
raw_input_data_s3_uri = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(raw_input_data_s3_uri)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/


In [49]:
execution = pipeline.start(
    parameters=dict(
#        InputDataUrl=raw_input_data_s3_uri,
#        ProcessingInstanceType="ml.c5.xlarge", # TODO:  This instance type is currently-hardcoded
#        ModelApprovalStatus="Approved",
    )
)

In [50]:
execution.wait()

WaiterError: Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state

In [ ]:
execution.list_steps()

# Approve the Model


The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

_Note:  You can do this within SageMaker Studio, as well.  However, we are approving programmatically in this case._

In [ ]:
model_package_arn=None

for execution_step in execution.list_steps():
    if execution_step["StepName"] == "RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break

print(model_package_arn)

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
)

print(model_package_update_response)

# Download the Trained Model

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## Download the Model

In [2]:
!aws s3 cp s3://$bucket/$sagemaker_project_id/TrainBERTModel/pipelines-0tssptovddj8-TrainBERTModel-5igoNLoKiW/output/model.tar.gz ./

download: s3://sagemaker-us-east-1-231218423789/project-dsoaws-p-ibxfjw9nuim7/TrainBERTModel/pipelines-0tssptovddj8-TrainBERTModel-5igoNLoKiW/output/model.tar.gz to ./model.tar.gz


In [5]:
!tar -C ./tmp -zxvf model.tar.gz

tensorboard/
transformers/
transformers/fine-tuned/
transformers/fine-tuned/config.json
transformers/fine-tuned/tf_model.h5
code/
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00000-of-00001
tensorflow/saved_model/0/assets/
tensorflow/saved_model/0/saved_model.pb


## Download the sourcedir.tar.gz

In [6]:
!aws s3 cp s3://$bucket/tensorflow-training-2020-12-22-02-18-02-763/source/sourcedir.tar.gz .

download: s3://sagemaker-us-east-1-231218423789/tensorflow-training-2020-12-22-02-18-02-763/source/sourcedir.tar.gz to ./sourcedir.tar.gz


In [7]:
!tar -C ./tmp -zxvf sourcedir.tar.gz

tf_bert_reviews.py


\# Get the Staging Model Endpoint

In [2]:
endpoint_name_staging='{}-staging'.format(sagemaker_project_name)

# _Wait Until the Staging Endpoint is Deployed_


In [ ]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name_staging)

# _Wait Until the ^^ Staging Endpoint ^^ is Deployed_

## Run A Sample Prediction

In [5]:
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name=endpoint_name_staging,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0)

In [6]:
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

[Predicted Star Rating: 5] This is great!


## Approve Staging Model for Production

In [ ]:
#https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-dsoaws-p-krqhvy8kqs0b-modeldeploy/view?region=us-east-1

In [7]:
# project_id='dsoaws-p-krqhvy8kqs0b'

In [17]:
from IPython.core.display import display, HTML

display(HTML('<b>Approve Model for Production in <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-modeldeploy/view?region={}">CodePipeline</a></b>'.format(sagemaker_project_id, region)))

## Get PROD Endpoint Name

In [11]:
endpoint_name_prod='{}-prod'.format(sagemaker_project_name)

# _Wait Until the Production Endpoint is Deployed_

In [ ]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name_prod)

# _Wait Until the ^^ Production Endpoint ^^ is Deployed_

In [12]:
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name=endpoint_name_prod,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0)

In [13]:
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

[Predicted Star Rating: 5] This is great!
